In [ ]:
'''
Author: Conor Lane, February 2024
Email: conor.lane1995@gmail.com

Takes all of the trials for a given cell's deconvolved activity and computes a single mean value that can be used as a relative measure of cell activity. 
Input, dell dictionary from epoch_recording.py, with ['Active'] Boollean obtained from running find_active_cells_ANOVA.py

'''

In [134]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as spc
from mpl_toolkits.mplot3d import Axes3D

n_baseline_frames = 5

In [135]:
def get_active_cells(deconvolved_traces):

    # going to return a dictionary with only active cells, formatted exactly the same as traces

    d = dict.fromkeys(deconvolved_traces.keys())

    for cell in deconvolved_traces:
        if deconvolved_traces[cell]['active'] == True:
            d[cell] = deconvolved_traces[cell]
        else:
            d.pop(cell,None)

    return d

In [137]:
# Input the filepath of the cell dictionary.  Opens the dict and displays the keys for an example cell. 
cell = "F:/Two-Photon/Psilocybin Project/Evoked Cohort Mice/ID237/12122022_GCaMP6s_ID237_psilo/TSeries-12122022-1307-033/suite2p/plane0/cells.pkl"

with open(cell,'rb') as f:
    cell_dict_1 = pickle.load(f)

print(cell_dict_1.keys())

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 119, 120, 121, 124, 125, 126, 128, 129, 130, 131, 132, 135, 136, 137, 138, 139, 140, 142, 143, 145, 146, 147, 149, 153, 154, 155, 156, 158, 159, 161, 162, 163, 167, 169, 170, 171, 172, 173, 174, 176, 177, 178, 179, 180, 181, 182, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 202, 204, 205, 206, 207, 208, 209, 210, 212, 213, 217, 218, 219, 220, 221, 224, 225, 226, 227, 229, 230, 231, 232, 234, 235, 237, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 251, 253, 255, 256, 259, 261, 262, 264, 265, 267, 271, 274, 27

In [138]:
# Get only the cells in the dictionary where Boollean "Active == True"
active = get_active_cells(cell_dict_1)
print(len(active.keys()))

# Get Information about the active cells. Returns Cell ID's in a list, Prints Presented Frequencies and Intensities, and the number of trials presented
# per combination. 
cell_list = list(active.keys())
nfreq = list(active[cell_list[0]]['deconvolved_traces'].keys())
nInt = list(active[cell_list[0]]['deconvolved_traces'][nfreq[0]].keys())
ntrial = list(active[cell_list[0]]['deconvolved_traces'][nfreq[0]][nInt[0]].keys())
print(nfreq, nInt, ntrial)

# For each cell, extracts the trial matrix and baseline corrects all the trials (computes a single baseline from all trial pre-stim periods and subtracts)
# Stores this in matrix_all_corrected.
matrix_all_corrected = []
for cell in active.keys():
    matrix_cell = np.array([[[active[cell]['deconvolved_traces'][i][j][k] for k in ntrial] for j in nInt] for i in nfreq])
    baseline = np.average(matrix_cell[:,:,:,:n_baseline_frames])
    baseline_subtracted = matrix_cell - baseline
    matrix_all_corrected.append(baseline_subtracted)

matrix_all_corrected = np.array(matrix_all_corrected)

# Flattens the matrix so that all trials are arrayed out equally -  takes shape (nFrames_total x NFrames_trial e.g. 7200,15)
matrix_flatten = (matrix_all_corrected).reshape(-1,len(matrix_all_corrected[:,0,0,0,0]))    

#
means = []
for i in range(0,len(matrix_flatten[0,:])):
    deconvolved_mean = np.mean(matrix_flatten[:,i])
    means.append(deconvolved_mean)

print(np.mean(means))




82
[4364, 5371, 6612, 8140, 10020, 12335, 15184, 18691, 23009, 28324, 34867, 42922] [35, 50, 65, 80] [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
0.27760914967277794
